## Homework 5: FX Carry Trade Strategy
### FINM 33150 (Winter 2025) - Quantitative Trading Strategies
#### NAME: Arturo Charleston
#### Student ID: 12449584

## Introduction
In the notebook I perform a carry traded based on the GBP. 

In [1]:
# PACKAGES
import os
import warnings
import nasdaqdatalink
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyprojroot import here
from fredapi import Fred

# Global Options
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.5f}'.format

# Set directory
home_directory = "/Users/charleston/Downloads/git_repositories/finm-qts-2025"
os.chdir(home_directory)

# KEYS
from config import FRED_KEY
from config import QUANDL_KEY

nasdaqdatalink.ApiConfig.api_key = QUANDL_KEY

In [2]:
# FUNCTIONS
def fetch_fred(fred_key, series_ids, output_dir='./data'):
    """
    Fetches data from FRED API and saves it to CSV files.

    Parameters:
    fred_key (str): API key for FRED.
    series_ids (dict): Dictionary with category as key and series ID as value.
    output_dir (str): Directory to save the CSV files. Default is './data'.

    Returns:
    None
    """
    fred = Fred(api_key=fred_key)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for category, series_id in series_ids.items():
        data = fred.get_series(series_id)
        data = data.reset_index()
        data.columns = ['date', 'value']
        data.to_csv(f'{output_dir}/{series_id}.csv', index=False)

        data_old = fred.get_series_first_release(series_id)
        data_old = data_old.reset_index()
        data_old.columns = ['date', 'value']
        data_old.to_csv(f'{output_dir}/{series_id}_first_release.csv', index=False)

## Data

In [3]:
# Fetch CPI data from FRED
series_ids = {
    'SONIA': 'IUDSOIA',  #  Daily Sterling overnight index average (SONIA) rate
}

fetch_fred(FRED_KEY, series_ids, output_dir='./data/FRED')
SONIA = pd.read_csv('./data/FRED/IUDSOIA.csv').set_index('date')
SONIA.columns = ['SONIA']

In [9]:
currencies = ["GBP", "TRY", "PKR", "CRC", "ZAR"]

currencies_data = pd.concat(
    [nasdaqdatalink.get_table('EDI/CUR', code=currency).set_index('date') for currency in currencies]
).sort_index().pivot(columns='code', values='rate').ffill()

currencies_data = currencies_data.loc[currencies_data.dropna(how='any', axis=0).index]

In [11]:
yield_currencies = ['Rep South Africa', 'Rep Turkey', 'Rep Costa Rica', 'Islamic Rep Pakistan']

yield_curves = (
    pd.read_csv(here('./data/InferredYieldCurves_2025-02-04.tab'), sep='\t')
    .query('shortname in @yield_currencies')
    .sort_values(by=['date', 'shortname', 'tenor'])
    .pivot_table(index=['date', 'tenor'], columns='shortname', values='yield')
    .rename(columns={'Rep South Africa': 'ZAR', 'Rep Turkey': 'TRY', 'Rep Costa Rica': 'CRC', 'Islamic Rep Pakistan': 'PKR'})
    .reset_index().set_index('date')
)

yield_curves.head(5)

shortname,tenor,PKR,CRC,ZAR,TRY
date,,,,,
2010-01-04,10Y,0.14872,0.06032,0.05996,0.06505
2010-01-04,1Y,0.08150,0.01275,0.01354,0.01656
2010-01-04,5Y,0.13650,0.04546,0.04530,0.05062
2010-01-04,6M,0.11647,0.01149,0.00940,0.01060
2010-01-05,10Y,0.14792,0.06095,0.05829,0.06348


In [5]:
yield_currencies = ['Rep South Africa', 'Rep Turkey', 'Rep Costa Rica', 'Islamic Rep Pakistan']
yield_curves = pd.read_csv(here('./data/InferredYieldCurves_2025-02-04.tab'), sep='\t')[['shortname', 'tenor', 'date', 'yield']]
yield_curves.query('shortname in @yield_currencies', inplace=True)
yield_curves.sort_values(by=['date', 'shortname', 'tenor'], inplace=True)
yield_curves = yield_curves.pivot_table(index=['date', 'tenor'], columns='shortname', values='yield').reset_index()

shortname_to_currency = {
    'Rep South Africa': 'ZAR',
    'Rep Turkey': 'TRY',
    'Rep Costa Rica': 'CRC',
    'Islamic Rep Pakistan': 'PKR'
}


# Rename the columns in yield_curves_wide
yield_curves.rename(columns=shortname_to_currency, inplace=True)
yield_curves.set_index('date', inplace=True)

yield_curves.head(5)

shortname,tenor,PKR,CRC,ZAR,TRY
date,,,,,
2010-01-04,10Y,0.14872,0.06032,0.05996,0.06505
2010-01-04,1Y,0.08150,0.01275,0.01354,0.01656
2010-01-04,5Y,0.13650,0.04546,0.04530,0.05062
2010-01-04,6M,0.11647,0.01149,0.00940,0.01060
2010-01-05,10Y,0.14792,0.06095,0.05829,0.06348
